# Yea or Nay?

## Sentiment analysis of 25,000 movie reviews in Python using a built-from-scratch Naive Bayes classifier

### Greg DeVore
### September 29th, 2017

This paper explores the use of sentiment analysis to classify movie reviews as being either "positive" (the reviewer enjoyed the movie) or "negative" (the reviewer did not enjoy the movie). This will be done by training two different classification models using reviews that are known to be either positive or negative, and testing the models using new reviews, whose sentiment is also known. The effectiveness of the models will be evaluated by looking at their confusion matrices, and also by calculating their accuracy, sensitivity, and specificity. The overall runtime associated with each model will also be considered.

The database of reviews used are from the following source:

Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis [http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf]. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).

The training and testing sets are balanced, meaning they contain the same number of reviews. The training set contains 12,500 positive reviews and 12,500 negative reviews, and the testing set contains an additional 12,500 positive reviews and 12,500 negative reviews.

Naive Bayes classifiers will be used as the models, both of which will be built from scratch in Python. The first uses a multinomial model to compute the likelihood, and the second uses a Bernoulli model. The theory behind these and their differences will be explored in subsequent sections. 

After training and testing the classifiers, it will be found that the Bernoulli Naive Bayes classifier performs slightly better than the multinomial model in terms of accuracy, sensitivity, and specificity. However, the advantages are minimal, and the lightweight nature and faster runtime associated with the multinomial model more than make up for the slight loss in accuracy. Overall, the multinomial model is a more attractive option for deployment in an actual use scenario.

## Theoretical Background

Recall that Bayes theorem gives us the conditional probability of a certain event, which in this case is the probabilty that a given movie review (represented by a vector of words denoted $x$) belongs to a particular class ($C_{k}$, where k represents the collection of either positive or negative reviews). The theorem for our situation is

$$P(C_k|x) = \frac{P(x \, | \, C_k)P(C_k)}{P(x)}$$

The left hand side term is the conditional probability we are interested in, namely the probability of a given movie review belonging to the positive or negative class. It is also known as the posterior probability. In terms of our problem, using the positive review class as an example, we could write this as

$$P(positive \, | \, review) = \frac{P(review \, | \, positive)P(positive)}{P(review)}$$

The right hand side terms will be used to calculate this value. They are:
- $P(review \, | \, positive)$. This is the likelihood of observing a given review, given that it is positive (or in general, of a given class). The computation of this term is precisely what changes when we switch from the multinomial model to the Bernoulli model.
- $P(positive)$. This is the prior probability associated with positive reviews. This is simply the number of positive reviews divided by the total number of reviews. The same approach can also give us $P(negative)$.
- $P(review)$. This is the prior probability associated with the current review. This is constant for all classes, and can be ignored in this particular problem.

We'll use the following equaiton to ultimately assign a review to a particular class:

$c = argmax_{k} P(C_k \, | \, x)$

The assigned class $(c)$ is found by calculating the posterior probability for each of the $k$ classes (we only have two) and choosing the class with the highest probability. Plugging in the right hand side of Bayes theorem (ignoring the denominator) gives

$c = argmax_{k} P(x \, | \, C_k)P(C_k)$

Now, we just need to calculate the likelihood for each review, given the particular class, and multiply it by the prior probability for that class. To actually calculate this, we'll make a relatively big assumption, in fact it's what gives the Naive Bayes classifier its "Naive" title. We'll assume each word in a review is independent of every other word, and treat each review as an unordered collection or "bag" of words. For this reason, this approach is known as the "bag of words" approach to classification.



### Multinomial Likelihood Model

Using the multinomial model, and the naive assumption about the indepence of words in a review, the above equation becomes

$c = argmax_{k} \big[ P(C_k) \prod_{w_i \in D} P(w_i \, | \, C_k) \big]$

We have replaced the likelihood term with a product sum over the likelihood of each word $w_i$ in a given review $D$, given a particular class $k$. To avoid underflow errors (numbers so small they are rounded to zero), we'll apply the log function in order to convert the product to a sum.

$c = argmax_{k} \big[ log\big(P(C_k)\big) + \sum_{w_i \in D} log\big(P(w_i \, | \, C_k)\big) \big]$

As discussed before, $P(C_k)$ is simply the ratio of the number of reviews belonging to class $k$ to the total number of reviews

$P(C_k) = \frac{N_k}{N_T}$

Here, $N_k$ is the number of reviews belonging to class $k$, and $N_T$ is the total number of reviews. The likelihood term associated with a particular word $w_i$, is

$P(w_i \, | \, C_k) = \frac{count(w_i,C_k) \, + \, 1}{count(w,C_k) \, + \, |V|}$

That is, the number of times the word appears in all reviews of class $k$ divided by the total number of words in all reviews of class $k$. 

The additional terms in the numerator and denominator are known as Laplace smoothing terms. The numerator term ensures that likelihoods of zero are never encountered, which could happen if a new word is seen that wasn't in the training set of reviews. The $V$ in the denominator represents the unique list of all words among all reviews, and the length of this list is added to balance out the addition of one to the occurence of every word and ensure the probabilities are properly normalized.

As an example, let's say we're trying to calculate the likelihood associated with the word "funny". It appears 80 times in all positive reviews, and only 40 times in all negative reviews. For simplicity, let's assume there are a total of 1000 words in all positive reviews, 1000 words in all negative reviews, and 500 unique words in all reviews. The likelihood is then:

$P(funny \, | \, positive) = \frac{80 + 1}{1000 + 500} = 0.054$

$P(funny \, | \, negative) = \frac{40 + 1}{1000 + 500} = 0.027$

Not surprisingly, the likelihood of observing the word "funny" in a postive review is higher than the likelihood of observing it in a negative review. The total likelihood for a given review is the sum of the log of the probabilities of every word in the review.

### Bernoulli Likelihood Model

In the multinomial model, we were concerned with how frequently a given word occurred among all words in a given class of reviews. In the Bernoulli model, we're more interested in how many different reviews a given word appears in, rather than how many times it appears. Using this model, each review is represented as a binary vector of length $V$ (recall that $V$ is the unique list of all words in all reviews) with a 1 in position $i$ if the $i^{th}$ word appears in that review, and a 0 otherwise. 

As an example, let's say we had two reviews:

$R_1$ = "Best movie I have ever seen"

$R_2$ = "I really disliked this movie"

The vector $V$ in this case is:

$ V = [ Best, movie, I, have, ever, seen, really, disliked, this] $

and each review would be represented as

$ x_1 = [ \enspace 1 \enspace 1 \enspace 1 \enspace 1 \enspace 1 \enspace 1 \enspace 0 \enspace 0 \enspace 0 \enspace ] $

$ x_2 = [  \enspace 0 \enspace 1 \enspace 1 \enspace 0 \enspace 0 \enspace 0 \enspace 1 \enspace 1 \enspace 1 \enspace ] $

The equation used to determine the assigned class is now

$c = argmax_{k} \big[ P(C_k) \prod_{i \in V} P(w_i \, | \, C_k)^{x_i}\big(1-P(w_i \, | \, C_k)\big)^\left({1-x_i}\right) \big]$

As with the multinomial model, we'll apply the log function to convert the product to a sum and avoid underflow errors

$c = argmax_{k} \big[ log\big(P(C_k)\big)  + \sum_{i \in V} log\big(P(w_i \, | \, C_k)^{x_i}\big(1-P(w_i \, | \, C_k)\big)^\left({1-x_i}\right)\big) \big]$

Once again, $P(C_k)$ is the prior probability associated with class $k$. Rather than iterate over the words in a single review, the sum for the likelihood function is now iterating over $V$, the unique list of all words in all reviews. The $x_i$ term is the $i^{th}$ entry from the binary vector representation of the review, which is 1 if the word appears in the review, and 0 otherwise. The likelihood term associated with a particular word $w_i$ is now

$P(w_i|C_k) = \frac{n_k(w_i) \, + \, 1}{N_k \, + \, 2}$

Where $n_k(w_i)$ is the number of reviews from class $k$ in which the word appears, and $N_k$ is the total number of reviews belonging to the class. As before, we're using Laplace smoothing terms to ensure no words have a probability of zero, and that the likelihoods are properly normalized.

### Differences Between Models

Note that the behavior of the Bernoulli model is fundamentally different from the multinomial model. Let's explore some of the biggest differences.

#### Presence of Words

In the multinomial model, the likelihood function iterates over each word in the current review. It uses the number of occurences of that word among all words in all reviews of a given class to determine the likelihood. It does not consider any words that do not occur in the current review.

In the Bernoulli model, the likelihood function iterates over each word in the entire vocabulary. If the $i^{th}$ word in the vocabulary is present in the current review, the second term becomes 1 and the probability of seeing that word in class $k$ is used $(P(w_i \, | \, C_k))$. If the $i^{th}$ word in the vocabulary is not present in the current review, the first term becomes 1 and the probability of not seeing that word in class $k$ is used $(1 - P(w_i \, | \, C_k))$. Unlike the multinomial model, the Bernoulli model accounts for both the presence and absence of words in a given review.

#### Encountering New Words

In the multinomial model, if a new word is encountered while classifying a review, the likelihood term for that word becomes

$P(w_i \, | \, C_k) = \frac{1}{count(w,C_k) \, + \, |V|}$

Which contributes a small, but nonzero, term to the overall likelihood for that review.

In the Bernoulli model, if a new word is encountered (one that is not in $V$), it is ignored. This is due to the fact that a binary vector of length $V$ is populated for each review, and there is no place for words that are not part of the training vocabulary. This is not a concern for this exercise, as only ~1% of the words in the set of test reviews are previously unseen.

## Python Implementation

With the theoretical background covered, we can now tackle the creation, training, and testing of these classifiers in Python. We'll start with defining functions to be used throughout the program. The first function is a decorator function for timing the main functions. The rest of the functions are:

- $\bf{returnWordsFromFile}$ : Accepts a file containing a single review as input and returns a scrubbed list of the relevant words from the review. These are words that contain letters only (no numbers), and have no special characters (such as brackets, apostrophes, or underscores).
- $\bf{buildLikelihoodDict}$: Iterates over all reviews in a directory and returns a dictionary of word counts. Each key is a unique word, and the associated value is the number of times that word appears among all reviews. This will aid in the calculation of the likelihood for a test review.
- $\bf{buildBinaryTermArray}$: Iterates over all reviews in a directory and returns a binary term vector for each review. Each term vector has an index for every word in the vocabulary generated from the training set of reviews. The term vector for each review will consist of 0's and 1's depending on which words are present in the review. Uses Numpy to create the binary vectors.
- $\bf{computePosteriorMultinomial}$: Iterates over all reviews in a directory and computes the posterior probability for each class using the multinomial model in order to determine the most likely sentiment (positive or negative). Uses the word count dictionary created previously to determine the likelihood of each word occuring in a positive or negative review. Uses the log of the probabilities to avoid underflow errors. Returns the percentage of reviews assigned to each class in order to determine the accuracy of the classifier.
- $\bf{computerPosteriorBernoulli}$: Iterates over all reviews in a directory and computes the posterior probability for each class using the Bernoulli model in order to determine the most likely sentiment (positive or negative). Creates a binary term vector for each review and uses the counts of words from the training set to determine the likelihood of each word occuring in a positive or negative review. Uses the log of the probabilities to avoid underflow errors. Uses Numpy to vectorize likelihood calculations and significantly reduce runtime. Returns the percentage of reviews assigned to each class in order to determine the accuracy of the classifier.

In [1]:
import os
import re
import math
import numpy
import time


# Decorator for timing function calls
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print '    %r,  %2.2f sec' % \
              (method.__name__, te-ts)
        return result
    return timed


# Function to return cleaned, relevant words from a review
def returnWordsFromFile(file):
    p = re.compile(r'[a-zA-z]+')
    ch = '`^_[]'
    fid = open(file, 'r')
    words = p.findall(fid.read())
    words = [x.lower() for x in words]
    words = [x.translate(None, ch) for x in words]
    words = [w for w in words if len(w) > 1]
    fid.close()
    return words


# Function for counting occurrences of words for multinomial model
@timeit
def buildLikelihoodDict(directory, files):
    word_dict = {}
    total_count = 0
    for item in files:
        words = returnWordsFromFile(os.path.join(directory, item))
        total_count += len(words)
        for w in words:
            if w in word_dict:
                word_dict[w] += 1
            else:
                word_dict[w] = 1
    return word_dict, total_count


# Function for creating binary term matrix for Bernoulli model
@timeit
def buildBinaryTermArray(directory, files, V):
    # Initialize binary term array
    btarray = numpy.zeros(shape=(len(files), len(V)))
    for i in range(len(files)):
        words = returnWordsFromFile(os.path.join(directory, files[i]))
        # Populate binary term array with 1's where specific words occur
        for w in words:
            j = V[w]  # Grab column associated with current word
            btarray[i, j] = 1  # Populate with a 1
    return btarray


# Function to compute posterior probability for multinomial model
@timeit
def computePosteriorMultinomial(directory, files, priorPos, priorNeg,
                pos_word_dict, pos_word_count, neg_word_dict, neg_word_count, lenV):
    pos_total = 0
    neg_total = 0
    for item in files:
        # Start with prior probabilities
        pos_prob = math.log(priorPos)
        neg_prob = math.log(priorNeg)
        words = returnWordsFromFile(os.path.join(directory, item))
        for w in words:
            # Compute likelihood of word occurring in a positive review
            pos = float(pos_word_dict.get(w, 0) + 1) / (pos_word_count + lenV)
            pos_prob += math.log(pos)
            # Compute likelihood of word occurring in a negative review
            neg = float(neg_word_dict.get(w, 0) + 1) / (neg_word_count + lenV)
            neg_prob += math.log(neg)
        # Larger number (both will be negative) is 'winner'
        if pos_prob > neg_prob:
            pos_total += 1
        else:
            neg_total += 1

    # Calculate percentage of reviews classified as positive/negative
    percentPos = float(pos_total) / len(files)
    percentNeg = float(neg_total) / len(files)

    return percentPos, percentNeg


# Function to compute posterior probability for Bernoulli model
@timeit
def computerPosteriorBernoulli(directory, files, priorPos, priorNeg, pos_binary_sum, neg_binary_sum, V):
    pos_total = 0
    neg_total = 0
    for item in files:
        # Start with prior probabilities
        pos_prob = math.log(priorPos)
        neg_prob = math.log(priorNeg)
        # Initialize binary term array
        btarray = numpy.zeros(len(V))

        words = returnWordsFromFile(os.path.join(directory, item))
        for w in words:
            if w in V:
                j = V[w]
                btarray[j] = 1

        # Compute positive likelihood using vectorization
        term1 = numpy.power(pos_binary_sum, btarray)
        term2 = numpy.power(1-pos_binary_sum, 1-btarray)
        pos_like = numpy.multiply(term1, term2)
        pos_like_log = numpy.log(pos_like)
        pos_prob += numpy.sum(pos_like_log)

        # Compute negative likelihood using vectorization
        term1 = numpy.power(neg_binary_sum, btarray)
        term2 = numpy.power(1-neg_binary_sum, 1-btarray)
        neg_like = numpy.multiply(term1, term2)
        neg_like_log = numpy.log(neg_like)
        neg_prob += numpy.sum(neg_like_log)

        # Larger number (both will be negative) is 'winner'
        if pos_prob > neg_prob:
            pos_total += 1
        else:
            neg_total += 1

    # Calculate percentage of reviews classified as positive/negative

    percentPos = float(pos_total) / len(files)
    percentNeg = float(neg_total) / len(files)

    return percentPos, percentNeg

The first step is to identify the repositories of training reviews and calculate the prior probability associated with each class. This is simply the number of positive or negative reviews divided by the total number of reviews. In this case, because the classes are balanced, the prior probability for each class is exactly 0.5.

In [2]:
pos_train_directory = '/Users/gdevore21/Documents/Projects/Independent/Movie Reviews/train/pos'
pos_files_train = os.listdir(pos_train_directory)
print('Using %i positive training reviews' % len(pos_files_train))

neg_train_directory = '/Users/gdevore21/Documents/Projects/Independent/Movie Reviews/train/neg'
neg_files_train = os.listdir(neg_train_directory)
print('Using %i negative training reviews' % len(neg_files_train))

# Calculate prior probabilities (percentage of pos/neg reviews)
priorPos = float(len(pos_files_train))/(len(pos_files_train) + len(neg_files_train))
priorNeg = float(len(neg_files_train))/(len(pos_files_train) + len(neg_files_train))

print
print('Prior probability for positive reviews = %f' % priorPos)
print('Prior probability for negative reviews = %f' % priorNeg)

Using 12500 positive training reviews
Using 12500 negative training reviews

Prior probability for positive reviews = 0.500000
Prior probability for negative reviews = 0.500000


Next, we'll process the reviews for each class and build the word count dictionaries for use in the multinomial model. We'll need the counts associated with each word in the training set of reviews and the total number of words for each class.

In [3]:
# Read reviews
print('Reading positive reviews...')
(pos_word_dict, pos_word_count) = buildLikelihoodDict(pos_train_directory, pos_files_train)
print('Done, %i unique words logged, %i words total' % (len(pos_word_dict.keys()), pos_word_count))
print('Reading negative reviews...')
(neg_word_dict, neg_word_count) = buildLikelihoodDict(neg_train_directory, neg_files_train)
print('Done, %i unique words logged, %i words total' % (len(neg_word_dict.keys()), neg_word_count))

Reading positive reviews...
    'buildLikelihoodDict',  12.16 sec
Done, 54769 unique words logged, 2864122 words total
Reading negative reviews...
    'buildLikelihoodDict',  11.97 sec
Done, 53256 unique words logged, 2790476 words total


In order to create the vocabulary array $V$, we can combine the keys of the positive and negative word count dictionaries. This will be used for both Laplace smoothing in the multinomial model, and to build the Bernoulli binary term arrays.

In [4]:
# Create V, unique list of all words in all reviews
print('Creating vocabulary vector V...')
combined_words = pos_word_dict.copy()
combined_words.update(neg_word_dict)
V = combined_words.keys()
V.sort()
print('There are %i unique words in all reviews' % len(V))
# Note position of ith word using a dictionary for population of binary vector
Vdict = {}
for i in range(len(V)):
    Vdict[V[i]] = i

Creating vocabulary vector V...
There are 73304 unique words in all reviews


With the unique vocabulary logged, we can create the binary term arrays for all positive and negative reviews. Summing these arrays along the columns gives us the number of documents that each word occurs in, which are required by the Bernoulli likelihood function.

In [5]:
# Create binary matrix for word occurrence in positive/negative reviews
print('Creating positive binary term matrix...')
pos_binary = buildBinaryTermArray(pos_train_directory, pos_files_train, Vdict)
# Calculate sums (number of documents containing each term)
pos_binary_sum = (numpy.sum(pos_binary, axis=0) + 1) / (len(pos_files_train) + 2)
print('Creating negative binary term matrix...')
neg_binary = buildBinaryTermArray(neg_train_directory, neg_files_train, Vdict)
# Calculate sums (number of documents containing each term)
neg_binary_sum = (numpy.sum(neg_binary, axis=0) + 1) / (len(neg_files_train) + 2)

Creating positive binary term matrix...
    'buildBinaryTermArray',  12.11 sec
Creating negative binary term matrix...
    'buildBinaryTermArray',  13.01 sec


Note that the creation of the binary term arrays took almost several times longer than the creation of the word count dictionaries. 

With all components created, we can now test our models. First, we'll point to the test directories. Note that as with the training set, we're also using a balanced test set with equal numbers of positive and negative reviews.

In [6]:
pos_test_directory = '/Users/gdevore21/Documents/Projects/Independent/Movie Reviews/test/pos'
pos_files_test = os.listdir(pos_test_directory)
print('Using %i positive testing reviews' % len(pos_files_test))
                                                 
neg_test_directory = '/Users/gdevore21/Documents/Projects/Independent/Movie Reviews/test/neg'
neg_files_test = os.listdir(neg_test_directory)
print('Using %i negative testing reviews' % len(neg_files_test))

Using 12500 positive testing reviews
Using 12500 negative testing reviews


Next, we'll run and time the multinomial model and calculate the terms of the confusion matrix along with the accuracy, PPV and NPV. The confusion matrix contains the follwing terms:

- True Positive Rate: Percentage of positive reviews that were correctly classified as positive.
- True Negative Rate: Percentage of negative reviews that were correctly classified as negative.
- False Positive Rate: Percentage of negative reviews that were incorrectly classified as positive.
- False Negative Rate: Percentage of positive reviews that were incorrectly classified as negative.

As for the remaining terms, The accuracy is:

$Accuracy = \frac{True \, Positive \, + \, True \, Negative}{Total \, Number \, of \, Reviews}$

In other words, how many test reviews were correctly classified out of the total number of test reviews. The sensitivity and specificity are:

$Sensitivity = \frac{True \, Positive}{True \, Positive \, + \, False \, Negative}$

$Specificity = \frac{True \, Negative}{True \, Negative \, + \, False \, Positive}$

The sensitivity is the number of correctly identified positive reviews out of all potential positive reviews (false negatives should actually be classified as positive). Similarly, the specificity is the number of correctly identified negative reviews out of all potential negative reviews (false positives should actually be classified as negative).

In [9]:
print('Testing reviews using multinomial model...')
# Positive reviews
(percentPos, percentNeg) = computePosteriorMultinomial(pos_test_directory, pos_files_test, priorPos, priorNeg,
                                        pos_word_dict, pos_word_count, neg_word_dict, neg_word_count, len(V))
multTP = percentPos  # Percent positive reviews
multFN = percentNeg  # Percent negative reviews
# Negative reviews
(percentPos, percentNeg) = computePosteriorMultinomial(neg_test_directory, neg_files_test, priorPos, priorNeg,
                                        pos_word_dict, pos_word_count, neg_word_dict, neg_word_count, len(V))
multTN = percentNeg  # Percent negative reviews
multFP = percentPos  # Percent positive reviews

# Calculate accuracy, sensitivity and specificity
multAcc = (multTP*len(pos_files_test) + multTN*len(neg_files_test))/(len(pos_files_test) + len(neg_files_test))
multSen = (multTP*len(pos_files_test))/(multTP*len(pos_files_test) + multFN*len(pos_files_test))
multSpc = (multTN*len(neg_files_test))/(multTN*len(neg_files_test) + multFP*len(neg_files_test))
print('Multinomial Accuracy: %f' % multAcc)
print('Multinomial Sensitivity: %f' % multSen)
print('Multinomial Specificity: %f' % multSpc)

Testing reviews using multinomial model...
    'computePosteriorMultinomial',  14.13 sec
    'computePosteriorMultinomial',  10.07 sec
Multinomial Accuracy: 0.815080
Multinomial Sensitivity: 0.749680
Multinomial Specificity: 0.880480


Before discussing the results, let's run and time the Bernoulli model and calculate the same terms.

In [10]:
print('Testing reviews using Bernoulli model...')
# Positive reviews
(percentPos, percentNeg) = computerPosteriorBernoulli(pos_test_directory, pos_files_test, priorPos, priorNeg,
                                        pos_binary_sum, neg_binary_sum, Vdict)
bernTP = percentPos  # Percent positive reviews
bernFN = percentNeg  # Percent negative reviews
# Negative reviews
(percentPos, percentNeg) = computerPosteriorBernoulli(neg_test_directory, neg_files_test, priorPos, priorNeg,
                                        pos_binary_sum, neg_binary_sum, Vdict)
bernTN = percentNeg  # Percent negative reviews
bernFP = percentPos  # Percent positive reviews

# Calculate accuracy, sensitivity and specificity
bernAcc = (bernTP*len(pos_files_test) + bernTN*len(neg_files_test))/(len(pos_files_test) + len(neg_files_test))
bernSen = (bernTP*len(pos_files_test))/(bernTP*len(pos_files_test) + bernFN*len(pos_files_test))
bernSpc = (bernTN*len(neg_files_test))/(bernTN*len(neg_files_test) + bernFP*len(neg_files_test))
print('Bernoulli Accuracy: %f' % bernAcc)
print('Bernoulli Sensitivity: %f' % bernSen)
print('Bernoulli Specificity: %f' % bernSpc)

Testing reviews using Bernoulli model...
    'computerPosteriorBernoulli',  85.03 sec
    'computerPosteriorBernoulli',  75.33 sec
Bernoulli Accuracy: 0.826680
Bernoulli Sensitivity: 0.769280
Bernoulli Specificity: 0.884080


## Summary and Conclusion

First, note that both models have an accuracy level of just over 80%. It is thought that humans are only able to agree on sentiment about 80% of the time, so this level of accuracy is acceptable for this type of problem. Also, both models have a higher true negative rate than true positive rate. Perhaps negative reviews use "stronger" language that make them easier to detect when compared to positive reviews.

The Bernoulli model showed a slightly higher level of accuracy, and most of that gain was due to a 2% increase in the true positive rate (both models have a similar true negative rate). Perhaps the fact that the Bernoulli model uses the entire vocabulary and can account for both the presence and absence of words in a given review gave it an advantage when it comes to positive sentiment. The increase in true positive rate also led to a reduction in false negative rate, which helped the Bernoulli model also achieve a slightly higher value for specificity. Overall, it seems that positive reviews were harder to correctly classify, perhaps due to the language used when expressing positive sentiment.

Another area to compare is the overall run time for both models. As noted earlier, the pre-processing time for the multinomial model is several times less than that required for the Bernoulli model. The word count dictionaries required for the multinomial model are less computationally intensive to create versus the binary term arrays required for each review in the Bernoulli model. More storage space is required for the binary term arrays, although sparse matrices could be used to reduce the amount of storage required for larger vocabularies. After having run both models on the test data, the multinomial model was able to classify the reviews an order of magnitude faster than the Bernoulli model. Once again, this has to do with the use of dictionaries in the multinomial model, which provide nearly constant time lookup regardless of their size. This is in direct contrast to the arrays used for the Bernoulli model, where runtime is proportional to the size of the array. Even though vectorized operations were used, the overall runtime still suffered because of the data structures used.

After considering both performance and overall runtime, the multinomial model emerges as the obvious choice for the ideal classifier to use should this experiment be moved forward to an actual production environment. The overall data structure is more lightweight, pre-processing time several times faster, and the time required to classify new reviews is roughly an order of magnitude less when compared to the Bernoulli model. Both models achieve similar levels of accuracy, and the slightly better performance seen by the Bernoulli model does not make up for the slower runtime and larger storage requirements.